In [345]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [346]:
xls = pd.ExcelFile('tiktok_data.xlsx')
print(xls.sheet_names)

['OrderSKUList']


In [347]:
df = pd.read_excel(xls, sheet_name='OrderSKUList', header=0)  # Header dimulai dari baris kedua
# df.head()

In [348]:
df = df[df['Cancelation/Return Type'] != 'Cancel']

In [349]:
df = df.drop(columns=['Cancelation/Return Type'])

In [350]:
df.shape

(47, 60)

In [351]:
# Drop kolom yang tidak diperlukan langsung pada df tanpa variabel baru
df.drop(columns=[
    'Order ID',
    'Order Status',
    'Order Substatus',
    'Normal or Pre-order',
    'SKU ID',
    'Seller SKU',
    'Variation',
    'Shipping Fee After Discount',
    'Original Shipping Fee',
    'Payment platform discount',
    'Buyer Service Fee',
    'Handling Fee',
    'Shipping Insurance',
    'Item Insurance',
    'Order Amount',
    'Order Refund Amount',
    'Paid Time',
    'RTS Time',
    'Shipped Time',
    'Delivered Time',
    'Cancelled Time',
    'Cancel By',
    'Cancel Reason',
    'Fulfillment Type',
    'Warehouse Name',
    'Tracking ID',
    'Delivery Option',
    'Shipping Provider Name',
    'Buyer Message',
    'Recipient',
    'Phone #',
    'Zipcode',
    'Country',
    'Districts',
    'Villages',
    'Detail Address',
    'Additional address information',
    'Payment Method',
    'Weight(kg)',
    'Product Category',
    'Package ID',
    'Seller Note',
    'Checked Status',
    'Checked Marked by',
    'Tokopedia Invoice Number',
    'Sku Quantity of return'
], inplace=True)


In [352]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47 entries, 0 to 58
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Product Name                    47 non-null     object
 1   Quantity                        47 non-null     int64 
 2   SKU Unit Original Price         47 non-null     object
 3   SKU Subtotal Before Discount    47 non-null     object
 4   SKU Platform Discount           47 non-null     object
 5   SKU Seller Discount             47 non-null     object
 6   SKU Subtotal After Discount     47 non-null     object
 7   Shipping Fee Seller Discount    47 non-null     object
 8   Shipping Fee Platform Discount  47 non-null     object
 9   Created Time                    47 non-null     object
 10  Buyer Username                  47 non-null     object
 11  Province                        47 non-null     object
 12  Regency and City                47 non-null     object
 1

In [353]:
df['Created Time'] = pd.to_datetime(df['Created Time'], format='%d/%m/%Y %H:%M:%S', dayfirst=True)


In [354]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47 entries, 0 to 58
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Product Name                    47 non-null     object        
 1   Quantity                        47 non-null     int64         
 2   SKU Unit Original Price         47 non-null     object        
 3   SKU Subtotal Before Discount    47 non-null     object        
 4   SKU Platform Discount           47 non-null     object        
 5   SKU Seller Discount             47 non-null     object        
 6   SKU Subtotal After Discount     47 non-null     object        
 7   Shipping Fee Seller Discount    47 non-null     object        
 8   Shipping Fee Platform Discount  47 non-null     object        
 9   Created Time                    47 non-null     datetime64[ns]
 10  Buyer Username                  47 non-null     object        
 11  Province     

In [355]:
def extract_variant(product_name):
    match = re.search(r'\b(Ares|Apollo|Athena)\b', product_name, re.IGNORECASE)
    return match.group(0) if match else None

# Terapkan fungsi ke kolom 'Nama Produk'
df['Product Name'] = df['Product Name'].apply(extract_variant)

In [356]:
df['Province'].unique()

array(['Jakarta', 'Banten', 'Kalimantan Timur', 'Jawa Barat', 'West Java',
       'Jawa Tengah', 'Sumatera Utara', 'Jakarta Province', 'DKI Jakarta',
       'Daerah Khusus Ibukota Jakarta', 'Kalimantan Utara'], dtype=object)

In [357]:
provinsi_koordinat = {
    'ACEH': (4.36855, 97.0253),
    'SUMATERA UTARA': (2.1153547, 99.5450974),
    'SUMATERA BARAT': (-0.7399397, 100.8000051),
    'RIAU': (0.2933466, 101.7068294),
    'JAMBI': (-1.6101232, 103.6131203),
    'SUMATERA SELATAN': (-3.0977119, 104.9142684),
    'BENGKULU': (-3.57785, 102.2655),
    'LAMPUNG': (-4.5585849, 105.4068),
    'KEPULAUAN BANGKA BELITUNG': (-2.7410513, 106.4405872),
    'KEPULAUAN RIAU': (3.9456514, 108.1428669),
    'DKI JAKARTA': (-6.2088, 106.8456),
    'JAWA BARAT': (-6.9248, 107.6071),
    'JAWA TENGAH': (-7.150975, 110.3499255),
    'DI YOGYAKARTA': (-7.797068, 110.3705293),
    'JAWA TIMUR': (-7.24917, 112.75083),
    'BANTEN': (-6.1202, 106.1505),
    'BALI': (-8.3405, 115.092),
    'NUSA TENGGARA BARAT': (-8.65293340, 117.36164760),
    'NUSA TENGGARA TIMUR': (-9.5018, 119.8256),
    'KALIMANTAN BARAT': (-0.0263, 109.3333),
    'KALIMANTAN TENGAH': (-1.6814878, 113.3823545),
    'KALIMANTAN SELATAN': (-3.092641, 114.603416),
    'KALIMANTAN TIMUR': (0.5247839, 116.9317887),
    'KALIMANTAN UTARA': (3.0166, 116.3319),
    'SULAWESI UTARA': (1.4538, 124.8917),
    'SULAWESI TENGAH': (-0.8375, 121.6255),
    'SULAWESI SELATAN': (-5.1476651, 119.4221),
    'SULAWESI TENGGARA': (-4.1434, 122.1746),
    'GORONTALO': (0.6371, 123.262),
    'SULAWESI BARAT': (-2.9521, 119.3793),
    'MALUKU': (-3.2385, 129.475),
    'MALUKU UTARA': (1.570999, 127.5101),
    'PAPUA': (-4.269928, 133.550003),
    'PAPUA BARAT': (-3.3333, 132.75),
    'PAPUA TENGAH': (-3.75, 137.0),
    'PAPUA PEGUNUNGAN': (-4.75, 140.0),
    'PAPUA SELATAN': (-6.75, 134.0),
    'PAPUA BARAT DAYA': (-0.875, 131.25),
    'NUSA TENGGARA BARAT (NTB)': (-8.652933, 116.5604),
    'BANGKA BELITUNG': (-2.7410513, 106.4405872),
    'KEPULAUAN BANGKA BELITUNG (BABEL)': (-2.7410513, 106.4405872)
}

# Dictionary untuk normalisasi nama provinsi yang berbeda
province_mapping = {
    'Jakarta': 'DKI JAKARTA',
    'Jakarta Province': 'DKI JAKARTA',
    'DKI Jakarta': 'DKI JAKARTA',
    'Daerah Khusus Ibukota Jakarta': 'DKI JAKARTA',
    'West Java': 'JAWA BARAT',
    'Jawa Barat': 'JAWA BARAT',
    'Central Java': 'JAWA TENGAH',
    'Jawa Tengah': 'JAWA TENGAH',
    'Yogyakarta': 'DI YOGYAKARTA',
    'East Java': 'JAWA TIMUR',
    'Jawa Timur': 'JAWA TIMUR',
    'Banten': 'BANTEN',
    'Bali': 'BALI',
    'West Sumatra': 'SUMATERA BARAT',
    'Sumatera Barat': 'SUMATERA BARAT',
    'North Sumatra': 'SUMATERA UTARA',
    'Sumatera Utara': 'SUMATERA UTARA',
    'Riau': 'RIAU',
    'South Sumatra': 'SUMATERA SELATAN',
    'Sumatera Selatan': 'SUMATERA SELATAN',
    'Bengkulu': 'BENGKULU',
    'Lampung': 'LAMPUNG',
    'Jambi': 'JAMBI',
    'West Nusa Tenggara': 'NUSA TENGGARA BARAT',
    'Nusa Tenggara Barat': 'NUSA TENGGARA BARAT',
    'East Nusa Tenggara': 'NUSA TENGGARA TIMUR',
    'Nusa Tenggara Timur': 'NUSA TENGGARA TIMUR',
    'West Kalimantan': 'KALIMANTAN BARAT',
    'Kalimantan Barat': 'KALIMANTAN BARAT',
    'Central Kalimantan': 'KALIMANTAN TENGAH',
    'Kalimantan Tengah': 'KALIMANTAN TENGAH',
    'South Kalimantan': 'KALIMANTAN SELATAN',
    'Kalimantan Selatan': 'KALIMANTAN SELATAN',
    'East Kalimantan': 'KALIMANTAN TIMUR',
    'Kalimantan Timur': 'KALIMANTAN TIMUR',
    'North Kalimantan': 'KALIMANTAN UTARA',
    'Kalimantan Utara': 'KALIMANTAN UTARA',
    'North Sulawesi': 'SULAWESI UTARA',
    'Sulawesi Utara': 'SULAWESI UTARA',
    'Central Sulawesi': 'SULAWESI TENGAH',
    'Sulawesi Tengah': 'SULAWESI TENGAH',
    'South Sulawesi': 'SULAWESI SELATAN',
    'Sulawesi Selatan': 'SULAWESI SELATAN',
    'Southeast Sulawesi': 'SULAWESI TENGGARA',
    'Sulawesi Tenggara': 'SULAWESI TENGGARA',
    'Gorontalo': 'GORONTALO',
    'West Sulawesi': 'SULAWESI BARAT',
    'Sulawesi Barat': 'SULAWESI BARAT',
    'Maluku': 'MALUKU',
    'North Maluku': 'MALUKU UTARA',
    'Maluku Utara': 'MALUKU UTARA',
    'Papua': 'PAPUA',
    'West Papua': 'PAPUA BARAT',
    'Papua Barat': 'PAPUA BARAT',
}

# Normalisasi nama provinsi
df['Normalized_Province'] = df['Province'].map(lambda x: province_mapping.get(x, x))

# Tambahkan kolom Latitude dan Longitude berdasarkan Provinsi yang sudah dinormalisasi
df['Latitude'] = df['Normalized_Province'].map(lambda x: provinsi_koordinat.get(x, (None, None))[0])
df['Longitude'] = df['Normalized_Province'].map(lambda x: provinsi_koordinat.get(x, (None, None))[1])

df.drop(columns=['Province'], inplace=True)

In [358]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47 entries, 0 to 58
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Product Name                    47 non-null     object        
 1   Quantity                        47 non-null     int64         
 2   SKU Unit Original Price         47 non-null     object        
 3   SKU Subtotal Before Discount    47 non-null     object        
 4   SKU Platform Discount           47 non-null     object        
 5   SKU Seller Discount             47 non-null     object        
 6   SKU Subtotal After Discount     47 non-null     object        
 7   Shipping Fee Seller Discount    47 non-null     object        
 8   Shipping Fee Platform Discount  47 non-null     object        
 9   Created Time                    47 non-null     datetime64[ns]
 10  Buyer Username                  47 non-null     object        
 11  Regency and C

In [359]:
desired_order_columns = [
    'Created Time', 
    'Product Name',
    'SKU Unit Original Price',
    'SKU Subtotal After Discount',
    'Quantity',
    'SKU Subtotal Before Discount',
    'SKU Seller Discount',
    'SKU Platform Discount',
    'Shipping Fee Seller Discount',
    'Shipping Fee Platform Discount',
    'Buyer Username',
    'Regency and City',
    'Normalized_Province',
    'Latitude',
    'Longitude',
    'Purchase Channel'
]
    
df = df[desired_order_columns + [col for col in df.columns if col not in desired_order_columns]]

In [360]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47 entries, 0 to 58
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Created Time                    47 non-null     datetime64[ns]
 1   Product Name                    47 non-null     object        
 2   SKU Unit Original Price         47 non-null     object        
 3   SKU Subtotal After Discount     47 non-null     object        
 4   Quantity                        47 non-null     int64         
 5   SKU Subtotal Before Discount    47 non-null     object        
 6   SKU Seller Discount             47 non-null     object        
 7   SKU Platform Discount           47 non-null     object        
 8   Shipping Fee Seller Discount    47 non-null     object        
 9   Shipping Fee Platform Discount  47 non-null     object        
 10  Buyer Username                  47 non-null     object        
 11  Regency and C

In [361]:
df.columns = ['Tanggal Order', 'Nama Produk', 'Harga Awal', 'Harga Setelah Diskon', 'Jumlah', 'Omzet', 'Diskon dari Seller', 'Diskon dari Platform', 'Voucher dari Seller', 'Voucher dari Platform', 'Username Buyer', 'Kota/Kabupaten', 'Provinsi', 'Latitude', 'Longitude', 'Channel']  

In [362]:
df['Tanggal Order'] = pd.to_datetime(df['Tanggal Order']).dt.date

In [363]:
df['Harga Awal'] = df['Harga Awal'].str.replace('IDR', '').str.replace('.', '')
df['Harga Awal'] = df['Harga Awal'].astype(float)

df['Harga Setelah Diskon'] = df['Harga Setelah Diskon'].str.replace('IDR', '').str.replace('.', '')
df['Harga Setelah Diskon'] = df['Harga Setelah Diskon'].astype(float)

df['Omzet'] = df['Omzet'].str.replace('IDR', '').str.replace('.', '')
df['Omzet'] = df['Omzet'].astype(float)

df['Diskon dari Seller'] = df['Diskon dari Seller'].str.replace('IDR', '').str.replace('.', '')
df['Diskon dari Seller'] = df['Diskon dari Seller'].astype(float)

df['Diskon dari Platform'] = df['Diskon dari Platform'].str.replace('IDR', '').str.replace('.', '')
df['Diskon dari Platform'] = df['Diskon dari Platform'].astype(float)

df['Voucher dari Seller'] = df['Voucher dari Seller'].str.replace('IDR', '').str.replace('.', '')
df['Voucher dari Seller'] = df['Voucher dari Seller'].astype(float)

df['Voucher dari Platform'] = df['Voucher dari Platform'].str.replace('IDR', '').str.replace('.', '')
df['Voucher dari Platform'] = df['Voucher dari Platform'].astype(float)

In [364]:
df['Omzet'].sum() - df['Diskon dari Seller'].sum() - df['Diskon dari Platform'].sum() - df['Voucher dari Seller'].sum() - df['Voucher dari Platform'].sum()

8127753.0

In [365]:
df['Tanggal Order'] = pd.to_datetime(df['Tanggal Order'])
df = df.sort_values(by='Tanggal Order', ascending=True)
df['Tanggal Order'] = df['Tanggal Order'].dt.date

In [366]:
df2 = df.copy()

In [367]:
start_date = df2['Tanggal Order'].min()
end_date = df2['Tanggal Order'].max()
all_dates = pd.date_range(start=start_date, end=end_date, freq='D')

In [368]:
# Menentukan rentang tanggal dari dataset
date_range = pd.date_range(start=df["Tanggal Order"].min(), 
                           end=df["Tanggal Order"].max(), 
                           freq='D')

# Membuat DataFrame baru dengan rentang tanggal lengkap
full_dates_df = pd.DataFrame({"Tanggal Order": date_range})

# Konversi kolom 'Tanggal Order' di df menjadi datetime
df["Tanggal Order"] = pd.to_datetime(df["Tanggal Order"])

# Menggabungkan dengan dataset utama untuk memastikan setiap tanggal ada dalam data
df_full = full_dates_df.merge(df, on="Tanggal Order", how="left")

# Mengisi nilai NaN dengan 0 untuk kolom jumlah produk yang terjual
df_full["Jumlah"] = df_full["Jumlah"].fillna(0)
df_full["Omzet"] = df_full["Omzet"].fillna(0)

In [369]:
df_full['Tanggal Order'] = pd.to_datetime(df_full['Tanggal Order']).dt.date

In [370]:
# Daftar varian parfum
varian_parfum = ["Ares", "Athena", "Apollo"]

# Membuat DataFrame dengan kombinasi semua tanggal dan varian parfum
full_dates_products = pd.MultiIndex.from_product(
    [df_full["Tanggal Order"].unique(), varian_parfum], 
    names=["Tanggal Order", "Nama Produk"]
).to_frame(index=False)

# Menggabungkan dengan dataset utama untuk memastikan setiap tanggal memiliki semua varian parfum
df_complete = full_dates_products.merge(df_full, on=["Tanggal Order", "Nama Produk"], how="left")

# Mengisi nilai NaN dengan 0 untuk kolom jumlah produk yang terjual
fill_columns = ["Jumlah", "Omzet", "Diskon dari Seller", "Diskon dari Platform"]
df_complete[fill_columns] = df_complete[fill_columns].fillna(0)

In [371]:
# Mengisi nilai NaN dengan dummy values yang diberikan
dummy_values = {
    'Harga Awal': 0.0,
    'Harga Setelah Diskon': 0.0,
    'Jumlah': 0,
    'Omzet': 0.0,
    'Diskon dari Seller': 0.0,
    'Diskon dari Platform': 0.0,
    'Voucher dari Seller': 0.0,
    'Voucher dari Platform': 0.0,
    'Username Buyer': 'dummy_user',
    'Kota/Kabupaten': 'dummy_city',
    'Provinsi': 'dummy_province',
    'Latitude': 0.0,
    'Longitude': 0.0,
    'Channel': 'TikTok'
}

df_complete.fillna(dummy_values, inplace=True)


In [372]:
df_complete['Tanggal Order'] = pd.to_datetime(df_complete['Tanggal Order']).dt.date

In [373]:
df_complete.to_excel('tiktok_cleaned.xlsx', index=False)

In [374]:
df_aggregated = df_complete.copy()

df_complete["Tanggal Order"] = pd.to_datetime(df_complete["Tanggal Order"]).dt.date

df_aggregated = df_complete.groupby(["Tanggal Order", "Nama Produk"])["Jumlah"].sum().reset_index()

date_product_pivot = df_aggregated.pivot(index="Tanggal Order", columns="Nama Produk", values="Jumlah").fillna(0)

date_product_pivot = date_product_pivot.reset_index()

date_product_pivot['Tanggal Order'] = pd.to_datetime(date_product_pivot['Tanggal Order']).dt.date # Ambil tanggal saja
# date_product_pivot['Marketplace'] = 'TikTok'

# Convert the pivoted DataFrame back to long format with only two columns
df_long_format = date_product_pivot.melt(id_vars=["Tanggal Order"], var_name="Nama Produk", value_name="Jumlah")
df_long_format['Marketplace'] = 'TikTok'
# Sort the DataFrame by date in ascending order
df_long_format = df_long_format.sort_values(by="Tanggal Order")
date_product_pivot['Marketplace'] = 'TikTok'


# export_path = "missing_date_filled.xlsx"
# df_fixed.to_excel(export_path, index=False)
# df_long_format.to_excel("long_format_data.xlsx", index=False)  

In [375]:
date_product_pivot.to_excel("tiktok_rekap.xlsx", index=False)